## DATA EXTRACTION

We load the data from the AEMET OpenData API, which is freely accessible.
* [AEMET OpenData](https://www.aemet.es/es/datos_abiertos/AEMET_OpenData)

Our source will be station **5783**, located at **Seville Airport** (IATA: SVQ).

From this database, we obtain the following parameters:

- Date of the measurement
- Indicative: ID of the weather station
- Name of the weather station
- Province
- Altitude in meters above sea level (MSNM)
- Average, maximum, and minimum temperature and their hours (Degrees Celsius)
- Precipitation (l/m²)
- Average wind speed (km/h)
- Wind direction (Numeric directions according to the compass rose)
- Maximum wind gust and its hour (km/h)
- Solar intensity (Ultraviolet Index)
- Maximum and minimum atmospheric pressure and their hour (hPa)

More information at https://www.aemet.es/es/eltiempo/observacion/ultimosdatos/ayuda

--- 

## EXTRACCIÓN DE DATOS

Cargamos los datos desde la API de AEMET OpenData, de acceso libre.
* [AEMET OpenData](https://www.aemet.es/es/datos_abiertos/AEMET_OpenData)

Nuestra fuente será la estación **5783**, situada en el **Aeropuerto de Sevilla** (IATA: SVQ).


De esta base de datos, obtenemos los siguientes parámetros:

- Fecha de la medición
- Indicativo: ID de la Estación meteorológica
- Nombre de la misma
- Provincia
- Altitud en MSNM
- Temperatura media, máxima, mínima y sus horas. (Grados celsius)
- Precipitaciones (l/m²)
- Velocidad media del viento (km/h)
- Dirección del viento (Rumbos numéricos según rosa de los vientos)
- Racha máxima de viento y su hora (km/h)
- Intensidad solar (Índice Ultravioleta)
- Presión atmosférica máxima, mínima y su hora (hPa)

Más información en https://www.aemet.es/es/eltiempo/observacion/ultimosdatos/ayuda

#### LIBRARIES

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import requests
import json
import sys
import warnings
import time
import datetime
from tqdm import tqdm
warnings.simplefilter(action='ignore', category=FutureWarning)

print("### --- Version --- ###")
print("Python:", sys.version)
print("Pandas:", pd.__version__)
print("### ----------------- ###")

### --- Version --- ###
Python: 3.11.7 (tags/v3.11.7:fa7a6f2, Dec  4 2023, 19:24:49) [MSC v.1937 64 bit (AMD64)]
Pandas: 2.2.3
### ----------------- ###


#### CONSTANTS

- API KEY: Personal key required to access AEMET OpenData API. You can request yours at [AEMET OpenData](https://opendata.aemet.es/)
- IDEMA: 5783 (Seville Airport)
- INTERVALS: 01/01/1951 to 30/06 of the last available year. (Typical delay 4 days)
- BASE_URL: We integrate everything inside

---

- API KEY: Clave personal necesaria para acceder a la API de AEMET OpenData. Puedes solicitar la tuya en [AEMET OpenData](https://opendata.aemet.es/)
- IDEMA: 5783 (Sevilla Aeropuerto)
- TRAMOS: 01/01/1951 a 30/06 del último año disponible. (Retardo habitual 4 días)
- URL_BASE: Integramos todo dentro



In [ ]:
API_KEY = os.getenv('AEMET_API_KEY')
IDEMA = 5783  # Sevilla Aeropuerto
URL_BASE = "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/{}T00:00:00UTC/fechafin/{}T23:59:59UTC/estacion/{}/?api_key={}"

TRAMOS_HISTORICO = [(f"{year}-01-01", f"{year}-06-30") for year in range(1951, 2024)]
TRAMO_2025 = [('2025-01-01', '2025-06-30')]


In [3]:
URL_TEST = f"https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/2024-01-01T00:00:00UTC/fechafin/2024-01-01T23:59:59UTC/estacion/{IDEMA}/?api_key={API_KEY}"

# Hacer la petición
response = requests.get(URL_TEST)

# Mostrar información
print("Status Code:", response.status_code)
print("Response Headers:", response.headers)
print("Response Content:", response.text)

if response.status_code == 200:
    json_response = response.json()
    
    # Extraer URLs
    data_url = json_response.get('datos')
    metadata_url = json_response.get('metadatos')

    if data_url:
        print("\nHaciendo petición a la URL de datos:", data_url)
        data_response = requests.get(data_url)
        print("Status Code (Datos):", data_response.status_code)
        print("Response Content (Datos):", data_response.text)

    if metadata_url:
        print("\nHaciendo petición a la URL de metadatos:", metadata_url)
        metadata_response = requests.get(metadata_url)
        print("Status Code (Metadatos):", metadata_response.status_code)
        print("Response Content (Metadatos):", metadata_response.text)

Status Code: 200
Response Headers: {'Date': 'Wed, 19 Feb 2025 13:15:36 GMT', 'Remaining-request-count': '149', 'Content-Length': '175', 'Connection': 'close', 'Content-Type': 'application/json;charset=ISO-8859-15', 'Set-Cookie': 'TS017a6cb0=0137e3526f9e8c3c9234b8d3f9a7af629fdd1d412a67960ba467af54014d55570708387cc8024aede5aa8efb124e1c78a45b50cfc5; Path=/; Domain=.opendata.aemet.es'}
Response Content: {
  "descripcion" : "exito",
  "estado" : 200,
  "datos" : "https://opendata.aemet.es/opendata/sh/51e42f9e",
  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"
}

Haciendo petición a la URL de datos: https://opendata.aemet.es/opendata/sh/51e42f9e
Status Code (Datos): 200
Response Content (Datos): [ {
  "fecha" : "2024-01-01",
  "indicativo" : "5783",
  "nombre" : "SEVILLA AEROPUERTO",
  "provincia" : "SEVILLA",
  "altitud" : "34",
  "tmed" : "12,1",
  "prec" : "0,0",
  "tmin" : "7,4",
  "horatmin" : "23:47",
  "tmax" : "16,8",
  "horatmax" : "14:40",
  "dir" : "27",
  "velmedi

#### MINING

Going through the sections (of four years so that it does not reach the limit of records per request), I insert each daily record in a dataframe. Once I have everything, I save it in a CSV file.

---
Recorriendo los tramos (de cuatro años para que no alcance el límite de registros por solicitud ), voy insertando cada registro diario en un dataframe. Al tenerlo todo, lo guardo en un archivo CSV.

In [4]:
def fetch_data(url):
    '''
    - Máximo de conexiones por minuto -> Parece ser que 25..
    '''
    response = requests.get(url)
    print(datetime.datetime.now().strftime("%H:%M:%S"))
    

    if response.status_code == 200:
        return response.json()
    elif response.status_code == 429:
        print(response)
        print(response.json().get('descripcion'))      
        wait_time = 60
        print(f"Retrying in {wait_time} seconds...")
        time.sleep(wait_time)
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

    # time.sleep(2.5)


In [5]:
# CONSULTA Y GUARDADO DE DATOS
def extract_data(tramos, filename):
    output = pd.DataFrame()
    metadata_saved = False

    # ITERACIÓN PARA CADA AÑO
    for f_inicio, f_fin in tqdm(tramos, file=sys.stdout, desc=f"Extracting {filename}..."):
        url_request = URL_BASE.format(f_inicio, f_fin, IDEMA, API_KEY)
        response = fetch_data(url_request)
        
        if response and 'datos' in response:
            data_url = response['datos']
            metadata_url = response.get('metadatos')
            
            # GUARDAR METADATOS SOLO LA PRIMERA VEZ
            if metadata_url and not metadata_saved:
                metadata = fetch_data(metadata_url)
                if metadata:
                    folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'raw'))
                    os.makedirs(folder, exist_ok=True)
                    metadata_path = os.path.join(folder, 'metadata.json')
                    with open(metadata_path, 'w', encoding='utf-8') as f:
                        json.dump(metadata, f, ensure_ascii=False, indent=4)
                    print(f"Metadata saved: {metadata_path}")
                    metadata_saved = True
            
            # Obtener datos
            data_interval = fetch_data(data_url)
            if data_interval:
                output = pd.concat([output, pd.DataFrame(data_interval)], ignore_index=True)
    
    folder = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'raw'))
    os.makedirs(folder, exist_ok=True)
    outpath = os.path.join(folder, filename)
    output.to_csv(outpath, sep=';', index=False)
    print(f"Dataset saved: {outpath}")


In [6]:
extract_data(TRAMOS_HISTORICO, 'datos_raw_1951-2024.csv')

Extracting datos_raw_1951-2024.csv...:   0%|          | 0/73 [00:00<?, ?it/s]14:16:49
14:16:49
Metadata saved: d:\Data Science\EDA_Easter_Climate\data\raw\metadata.json
14:16:49
Extracting datos_raw_1951-2024.csv...:   1%|▏         | 1/73 [00:00<00:26,  2.68it/s]14:16:50
14:16:50
Extracting datos_raw_1951-2024.csv...:   3%|▎         | 2/73 [00:00<00:26,  2.72it/s]14:16:50
14:16:50
Extracting datos_raw_1951-2024.csv...:   4%|▍         | 3/73 [00:01<00:23,  2.93it/s]14:16:50
14:16:51
Extracting datos_raw_1951-2024.csv...:   5%|▌         | 4/73 [00:01<00:23,  2.88it/s]14:16:51
14:16:51
Extracting datos_raw_1951-2024.csv...:   7%|▋         | 5/73 [00:01<00:23,  2.85it/s]14:16:51
14:16:51
Extracting datos_raw_1951-2024.csv...:   8%|▊         | 6/73 [00:02<00:22,  3.02it/s]14:16:51
14:16:51
Extracting datos_raw_1951-2024.csv...:  10%|▉         | 7/73 [00:02<00:21,  3.12it/s]14:16:52
14:16:52
Extracting datos_raw_1951-2024.csv...:  11%|█         | 8/73 [00:03<00:33,  1.94it/s]14:16:53
14:16:5

In [8]:
extract_data(TRAMO_2025, 'datos_raw_2025.csv')

Extracting datos_raw_2025.csv...:   0%|          | 0/1 [00:00<?, ?it/s]14:21:26
14:21:26
Metadata saved: d:\Data Science\EDA_Easter_Climate\data\raw\metadata.json
14:21:26
Extracting datos_raw_2025.csv...: 100%|██████████| 1/1 [00:00<00:00,  1.95it/s]
Dataset saved: d:\Data Science\EDA_Easter_Climate\data\raw\datos_raw_2025.csv
